In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(phyloseq)
library(plyr)
library(dplyr)
library(ggplot2)
library(rwantshue)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [4]:
%%R
#detach(package:dplyr)
#detach(package:plyr)

Error in detach(package:plyr) : invalid 'name' argument


In [9]:
%%R
physeq = import_biom("../../SeqData/ITS.otu_table.tax.meta.biom")

In [10]:
%%R
Merge<-paste(as.character(sample_data(physeq)$Trtmt),as.character(sample_data(physeq)$Month),as.character(sample_data(physeq)$Cosm),sep="_")
sample_data(physeq)$Merge <- Merge
# Creating a new column in the phyloseq sample data called Merge,
# which contains a concatenated ID so all samples from the same mineral, month, and cosm
# will have the same ID (thus merging PCR and buffer replicates).

ps.merged = merge_samples(physeq, "Merge")
# Merging the phyloseq object by biological replicates

keep=c("Cosm","Month","Trtmt")
sd = sample_data(ps.merged)
sd = sd[,keep]
sd$Trtmt = substring(row.names(sd),1,1)
sd$Trtmt[sd$Trtmt=="B"]="Blank"
sd$Trtmt[sd$Trtmt=="F"]="Ferrihydrite"
sd$Trtmt[sd$Trtmt=="Q"]="Quartz"
sd$Trtmt[sd$Trtmt=="H"]="Heavy Fraction"
sd$Trtmt[sd$Trtmt=="S"]="Soil"
sd$Trtmt[sd$Trtmt=="K"]="Kaolinite"
sample_data(ps.merged) = sd
physeq = ps.merged

In [21]:
%%R
EMF = read.csv("../../SeqData/EMF.txt",quote ="",header=FALSE)

In [22]:
%%R
m = c()
l = length(EMF)
for (i in c(1:l)) {
    m = c(m,paste(EMF[[i]]))
}

In [11]:
%%R
physeq

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 2200 taxa and 73 samples ]
sample_data() Sample Data:       [ 73 samples by 3 sample variables ]
tax_table()   Taxonomy Table:    [ 2200 taxa by 7 taxonomic ranks ]


In [23]:
%%R
ps = psmelt(physeq)
ps = ps %>%
    mutate(EMFv =ifelse(ps$Rank5 %in% m | ps$Rank6 %in% m, "EMF",
                ifelse(ps$Rank2=="p__Glomeromycota","AMF","Other")))
ps$EMFv[is.na(ps$EMFv)]="Other"
summary(ps$EMFv=="AMF")

   Mode   FALSE    TRUE    NA's 
logical  156220    4380       0 


In [14]:
%%R
ps2 = ps %>%
    group_by(Trtmt,Month,OTU)%>%
    filter(sum(Abundance)>0)%>%
    group_by(Trtmt,EMFv) %>%
    summarize(n=n_distinct(OTU))
ps2

Source: local data frame [14 x 3]
Groups: Trtmt

            Trtmt  EMFv    n
1           Blank Other    1
2    Ferrihydrite   AMF   44
3    Ferrihydrite   EMF    4
4    Ferrihydrite Other 1146
5  Heavy Fraction Other   24
6       Kaolinite   AMF    6
7       Kaolinite   EMF    1
8       Kaolinite Other  698
9          Quartz   AMF   29
10         Quartz   EMF    4
11         Quartz Other 1139
12           Soil   AMF   44
13           Soil   EMF    3
14           Soil Other 1375


In [15]:
%%R
ps3 = ps %>%
    group_by(Sample)%>%
    mutate(relabund=Abundance/sum(Abundance))%>%
    group_by(Sample,Trtmt,EMFv)%>%
    summarize(relabund=sum(relabund))%>%
    group_by(Trtmt,EMFv)%>%
    summarize(relabund=mean(relabund))

ps3

Source: local data frame [18 x 3]
Groups: Trtmt

            Trtmt  EMFv     relabund
1           Blank   AMF 0.000000e+00
2           Blank   EMF 0.000000e+00
3           Blank Other 1.000000e+00
4    Ferrihydrite   AMF 1.735825e-02
5    Ferrihydrite   EMF 5.102433e-03
6    Ferrihydrite Other 9.775393e-01
7  Heavy Fraction   AMF 0.000000e+00
8  Heavy Fraction   EMF 0.000000e+00
9  Heavy Fraction Other 1.000000e+00
10      Kaolinite   AMF 2.505739e-04
11      Kaolinite   EMF 6.165988e-05
12      Kaolinite Other 9.996878e-01
13         Quartz   AMF 5.181039e-03
14         Quartz   EMF 2.019924e-03
15         Quartz Other 9.927990e-01
16           Soil   AMF 3.161485e-03
17           Soil   EMF 5.449234e-05
18           Soil Other 9.967840e-01


In [19]:
%%R
ps.plot = ps %>%
    group_by(Sample)%>%
    filter(sum(Abundance)>1500)%>%
    mutate(relabund=Abundance/sum(Abundance))%>%
    group_by(Trtmt,Month,EMFv)

p = ggplot(ps.plot, aes(x = Trtmt,fill=EMFv))
p = p + geom_bar(stat="identity")
p = p + facet_grid(~Month)
p

RRuntimeError: Error in exists(name, envir = env, mode = mode) : 
  argument "env" is missing, with no default
